In [84]:
import pandas as pd
import openpyxl
import openpyxl.drawing.image
import base64
import json

def extract_images_from_xlsx(file_path):
    workbook = openpyxl.load_workbook(filename=file_path)
    images = []

    for sheet in workbook.worksheets:
        for drawing in sheet._images:
            if isinstance(drawing, openpyxl.drawing.image.Image):
                images.append(drawing)

    return images

def read_xls_with_images(file_path):
    df = pd.read_excel(file_path, keep_default_na=False)
    images = extract_images_from_xlsx(file_path)
    assert len(df) == len(images)
    df["image"] = images
    return df

In [85]:
df = read_xls_with_images('data.xlsx')

In [86]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL"),
)

In [95]:
prompt='''任务简介：
- 识别图片中的主要物品，判断其分类并评估新旧程度。

分类标准：
- 服装：大衣、皮衣、半袖、裤子、裙子、内衣裤、秋衣类、毛衣类、工装、校服。
- 家电：空气净化器、厨房家电、家居家电。
- 乐器：电子乐器、琴类。
- 玩具：小件（手办）、大件、毛绒玩具。
- 图书：儿童绘本、课外书、小说、套装书籍、课本。
- 手机：智能手机、功能机。
- 笔记本电脑：品牌电脑、非品牌电脑。

回答格式（JSON）：
{
    "exist": "是/否",
    "category": {
        "primary": "一级分类",
        "secondary": "二级分类"
    },
    "freshness": "新/旧/中等"
}
'''

def describe(b64image):
    response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": b64image,
                        "detail": "low",
                    }
                }
            ]
        }],
        max_tokens=300,
    )
    print(response.model_dump_json())
    return response

In [96]:
def process(image: openpyxl.drawing.image.Image):
    x = f"data:image/{image.format};base64,{base64.b64encode(image.ref.getvalue()).decode()}"
    response = describe(x)
    s = response.choices[0].message.content.lstrip("```json\n").rstrip("\n```")
    item = json.loads(s)
    exist = item["exist"]
    primary = secondary = freshness = ""
    if exist == "是":
        primary = item["category"]["primary"]
        secondary = item["category"]["secondary"]
        freshness = item["freshness"]
    return exist, primary, secondary, freshness, response.usage.total_tokens, json.dumps(item, ensure_ascii=False), response.model_dump_json()

In [97]:
out = process(df.loc[28, "image"])

{"id":"chatcmpl-8NAnnXuRnhfG7jrEDByQP0X4LcovD","choices":[{"finish_reason":null,"index":0,"message":{"content":"{\n    \"exist\": \"是\",\n    \"category\": {\n        \"primary\": \"玩具\",\n        \"secondary\": \"毛绒玩具\"\n    },\n    \"freshness\": \"中等\"\n}","role":"assistant","function_call":null,"tool_calls":null},"finish_details":{"type":"stop","stop":"<|fim_suffix|>"}}],"created":1700533759,"model":"gpt-4-1106-vision-preview","object":"chat.completion","system_fingerprint":null,"usage":{"completion_tokens":47,"prompt_tokens":376,"total_tokens":423}}


In [90]:
out

('是',
 '玩具',
 '毛绒玩具',
 '新',
 426,
 '{"exist": "是", "category": {"primary": "玩具", "secondary": "毛绒玩具"}, "freshness": "新"}',
 '{"id":"chatcmpl-8NAhSSZvBSwX8DwzSmr1Yb5xHzDD0","choices":[{"finish_reason":null,"index":0,"message":{"content":"```json\\n{\\n    \\"exist\\": \\"是\\",\\n    \\"category\\": {\\n        \\"primary\\": \\"玩具\\",\\n        \\"secondary\\": \\"毛绒玩具\\"\\n    },\\n    \\"freshness\\": \\"新\\"\\n}\\n```","role":"assistant","function_call":null,"tool_calls":null},"finish_details":{"type":"stop","stop":"<|fim_suffix|>"}}],"created":1700533366,"model":"gpt-4-1106-vision-preview","object":"chat.completion","system_fingerprint":null,"usage":{"completion_tokens":50,"prompt_tokens":376,"total_tokens":426}}')

In [92]:
def process1(x):
    return x,x,x,x

In [93]:
process1(df.loc[28, "序号"])

(29, 29, 29, 29)

In [94]:
df["序号"].map(process1)

0              (1, 1, 1, 1)
1              (2, 2, 2, 2)
2              (3, 3, 3, 3)
3              (4, 4, 4, 4)
4              (5, 5, 5, 5)
               ...         
236    (237, 237, 237, 237)
237    (238, 238, 238, 238)
238    (239, 239, 239, 239)
239    (240, 240, 240, 240)
240    (241, 241, 241, 241)
Name: 序号, Length: 241, dtype: object

In [ ]:
df["exist"], df["primary"], df["secondary"], df["freshness"], df["tokens"], df["json"], df["model"] = zip(*df["image"].map(process))